In [1]:
"""
/**
 * Implementation of the symmetric envy free cake-cutting procedure on a 1-dimensional cake
 *
 * @author Thomas Gibson
 * @date 30/04/2021
 */
"""


import numpy as np, numpy.random
import random as ran


from Agent import Agent


#This function will generate n values that will sum to exactly 1. 
#This will form an array showing how much an agent values a single atom
#This generating function could be changed to make use of other distributions if preferred
def generatevaluations(n): 
    x = np.random.random(n)
    x /= x.sum()
    return x


#This function returns the last atom included in a piece marked by an agent
def mark(agent):
    i = 0
    v = 0
    while v <= 0.5:
        v = v + agent.vals[i]
        i+=1
    lastpiece = (i-1)
    agent.mark = lastpiece
    return lastpiece

#This is the function that takes the marks, caculate the midpoint atom and designates the pieces resulting from the cut
def cut(x, y, numatoms):
    midpoint = int(((x+y)/2))
    piece1 = (0, midpoint)
    piece2 = (midpoint+1, numatoms-1)
    return piece1, piece2    

#This function will determine an agents value of a piece (vop)
def vop(agent, piece):
    value = 0
    a, b = piece   
    
    #This line is needed as the range function is not inclusive on the top end
    b = b+1
    
    for x in range(a, b):
        value = value + agent.vals[x]
    return value

#This function represents the choose portion of cut and choose
def choose(agent1, agent2, piece1, piece2):
    
    if(agent2.mark > agent1.mark):
        agent2.allocation = piece2
        agent1.allocation = piece1
    elif(agent1.mark > agent2.mark):
        agent1.allocation = piece2
        agent2.allocation = piece1   
    
    #Optional Step to help minimise envy freeness.
    #Normally we would simply allocate the pieces randomly if the marks were in the same place
    #However, we can simply use the agents valuations to help iminimise envy freeness generate when this is the case
    elif vop(agent2, piece1) > vop(agent2, piece2):
        agent1.allocation = piece2
        agent2.allocation = piece1
    elif vop(secondchooser, piece2) > vop(agent2, piece1):
        agent1.allocation = piece1
        agent2.allocation = piece2      
    
    return (agent1.allocation, agent2.allocation)


#This function determines if an allocation is envy free or not
def envyfreecheck(agent1, agent2):
    #This if statement determins if both agents consider their own piece to be greater than the other agents.
    #If this is the case, we can conclude the allocation is envy free, else it is not.
    if ((vop(agent1, agent1.allocation) - vop(agent1, agent2.allocation) >= 0) and (vop(agent2, agent2.allocation) - vop(agent2, agent1.allocation) >= 0)):
        return 1
    else:
        return 0
    
#It is also useful to know if the marks were on the same atom of cake
def markcheck(agent1, agent2):
    if agent1.mark == agent2.mark:
        return 1
    else:
        return 0

#This function returns 1 when an allocation is not envy free and they have the same mark
def nonenvyfreemarkchecker(agent1, agent2):
    if ((envyfreecheck(agent1, agent2) == 0) and (markcheck(agent1, agent2) == 1)):
        return 1
    else:
        return 0

#Function to print the final allocation for each agent and their value of the piece they recieved, and their percieved value of the other agent
def printallocation(agent1, agent2):
    print("Agent " ,agent1.name, "recieved atoms within the range" ,agent1.allocation)
    print("This agent obtained a value of " ,vop(agent1, agent1.allocation), ", and consider the other agent to have obtained", vop(agent1, agent2.allocation))
    print("Agent " ,agent2.name, "recieved atoms within the range" ,agent2.allocation)   
    print("This agent obtained a value of " ,vop(agent2, agent2.allocation), ", and consider the other agent to have obtained", vop(agent2, agent1.allocation))    
    if (envyfreecheck(agent1,agent2) == 1):
        print("This allocation was envy free")
    else:
        print("This allocation was not envy free")



    

In [2]:
#Determine how many atoms we want
atomsexample = 20

#Generate some agents
agentA = Agent("A", generatevaluations(atomsexample))
agentB = Agent("B", generatevaluations(atomsexample))


#Have the agents both mark the cake
agentAmark = mark(agentA)
agentBmark = mark(agentB)

#Cut the cake at the midpoint of the marks
piece1, piece2 = cut(agentAmark, agentBmark, atomsexample)

#Have agents select pieces
agentAallocation, agentBallocation = choose(agentA, agentB, piece1, piece2)


#Print out the range of pieces allocated to each agent and how they value the pieces and if the allocation was envy free.
printallocation(agentA, agentB)


Agent  A recieved atoms within the range (10, 19)
This agent obtained a value of  0.5363391905583471 , and consider the other agent to have obtained 0.46366080944165294
Agent  B recieved atoms within the range (0, 9)
This agent obtained a value of  0.555850254533688 , and consider the other agent to have obtained 0.444149745466312
This allocation was envy free


In [3]:
#Function that does a single iteration of the symmetric cut and choose
#with random agents and random cutter and chooser for a given number of agents.
#We return a value, 1 or 0 depending if the allocation was envy free or not
def iteratingsymmetric(atoms):
    agentA = Agent("A", generatevaluations(atoms))
    agentB = Agent("B", generatevaluations(atoms))
    agentAmark = mark(agentA)
    agentBmark = mark(agentB)
    piece1, piece2 = cut(agentAmark, agentBmark, atoms)
    agentAallocation, agentBallocation = choose(agentA, agentB, piece1, piece2)
    efck = envyfreecheck(agentA, agentB)
    nefmc = nonenvyfreemarkchecker(agentA, agentB)
    return (efck, nefmc)


In [4]:
#Iterate through 1000 Symmetric Cut and Choose Procedures and determine number of envy free iterations with a specified number of atoms.
#Here you can edit the number of iterations or atoms
iterations = 1000
numberofatoms = 1000
counteref = 0
counterefmc = 0

for m in range(1, iterations+1, 1):
    ef, efmck = iteratingsymmetric(numberofatoms)
    counteref = counteref + ef
    counterefmc = counterefmc + efmck

percentageef = round(((counteref/iterations)*100), 2)
percentageefmc = (counterefmc/(iterations-counteref)*100)

    
print("With",numberofatoms, "atoms and", iterations, "iterations, we get that a total of",counteref,"(",percentageef,"%) iterations that are envy free.")
print("A total of",counterefmc,"(",percentageefmc,"%) non-envy free iterations having the same marks")

With 1000 atoms and 1000 iterations, we get that a total of 966 ( 96.6 %) iterations that are envy free.
A total of 34 ( 100.0 %) non-envy free iterations having the same marks
